In [206]:
import numpy as np
import pickle  # 用 pickle 替代 cPickle
import gym
np.bool8 = np.bool_

In [207]:
import gym

In [208]:
env = gym.make("CliffWalking-v0")
observation, _ = env.reset()

In [209]:
# 0: move up

# 1: move right

# 2: move down

# 3: move left

In [210]:
observation,_

(36, {'prob': 1})

In [211]:
for i in range(4):
    observation, reward, terminated, truncated, info = env.step(0)  # 更新为新的返回值
    print(i,observation,reward, terminated,truncated)

0 24 -1 False False
1 12 -1 False False
2 0 -1 False False
3 0 -1 False False


In [212]:
from torch.distributions import Categorical

In [213]:
from torch import nn
import torch

In [214]:
class MLP(nn.Module):
    
    
    def __init__(self,output_dim):
        super().__init__()
        self.embeddings = nn.Embedding(48,200)
        self.linear1 = nn.Linear(200,200,bias=False)
        self.linear2 = nn.Linear(200,output_dim,bias=False)
        
       # Xavier 初始化 + 偏置归零
        nn.init.xavier_normal_(self.embeddings.weight)
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)

        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,idxs):
        x = self.embeddings(idxs)
        x = self.relu(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return self.softmax(x)
    
    def sample_action(self,x):
#         with torch.no_grad():
                # 添加探索机制
        if np.random.random() < 0.6:
            action = np.random.randint(0, 4)
            probs = self.forward(x).squeeze()
#             print(probs.shape,action)
            return torch.LongTensor([action]), torch.log(probs[action]+1e-8).unsqueeze(dim=-1)
        
        prob = self.forward(x)
        dist = Categorical(prob)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action,log_prob

In [215]:
mm = MLP(4)

In [216]:
mm.sample_action(torch.LongTensor([1]))

(tensor([1]), tensor([-1.4070], grad_fn=<UnsqueezeBackward0>))

In [217]:
def discount_rewards(rewards):
    ans = np.zeros_like(rewards)
    adding = 0
    for t in reversed(range(len(rewards))):
        if rewards[t] != 0:
            adding = 0
        adding = adding * 0.99 + rewards[t]
        ans[t] = adding
    return ans

In [218]:
model = MLP(4)

In [219]:
from torch.optim import RMSprop
import torch

In [220]:
optimizer = RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-5)

In [221]:
def get_base_reward(next_state, done):
    # 默认环境奖励
    if done:  # 到达终点
        return 10.0  # 原为0，调整为正向奖励
    elif is_cliff(next_state):  # 掉入悬崖
        return -100.0
    else:  # 普通移动
        return -1.0  # 保持原惩罚
def distance_reward(state):
    # 状态转换为坐标 (行, 列)
    row = state // 12
    col = state % 12
    # 终点坐标 (3, 11)
    target_row, target_col = 3, 11
    # 曼哈顿距离（横向距离优先）
    distance = (target_col - col) + 0.1 * abs(target_row - row)  # 横向距离权重更高
    return 0.5 * distance  # 系数可调
def safe_path_reward(state):
    row = state // 12
    # 若在第一行（索引0），给予额外奖励
    if row == 0:
        return 2.0  # 系数可调
    else:
        return 0.0
def safe_path_reward(state):
    row = state // 12
    # 若在第一行（索引0），给予额外奖励
    if row == 0:
        return 2.0  # 系数可调
    else:
        return 0.0
def progress_reward(state, prev_col):
    current_col = state % 12
    # 若向右移动，给予奖励
    if current_col > prev_col:
        return 1.0  # 系数可调
    else:
        return -0.5  # 向左或不动有轻微惩罚
    
def is_cliff(state: int) -> bool:
    """
    判断状态是否位于悬崖区域
    :param state: 当前状态编号（0到47之间的整数）
    :return: True（是悬崖） / False（非悬崖）
    """
    # 将状态转换为网格坐标（行、列）
    row = state // 12  # 行索引（0到3）
    col = state % 12   # 列索引（0到11）
    
    # 悬崖区域：第3行（索引2）的第1到10列
    return (row == 2) and (1 <= col <= 10)

class CustomCliffWalkingReward:
    def __init__(self):
        self.prev_col = 0  # 记录上一步的列坐标

    def reset(self):
        self.prev_col = 0

    def __call__(self, state, next_state, done):
        # 基础奖励
        base_reward = get_base_reward(next_state, done)
        
        # 距离奖励
        dist_r = distance_reward(next_state)
        
        # 安全路径奖励
        safe_r = safe_path_reward(next_state)
        
        # 进度奖励
        current_col = next_state % 12
        prog_r = progress_reward(next_state, self.prev_col)
        self.prev_col = current_col  # 更新列坐标
        
        # 合并奖励（加权求和）
        total_reward = base_reward + dist_r + safe_r + prog_r
        
        return total_reward

# 初始化奖励函数
reward_shaper = CustomCliffWalkingReward()

In [185]:
def sample_action(self,x):
#         with torch.no_grad():
            # 添加探索机制
    if np.random.random() < 0.1:
        action = np.random.randint(0, 4)
        probs = self.forward(x).squeeze()
#             print(probs.shape,action)
        return torch.LongTensor([action]), torch.log(probs[action]+1e-8).unsqueeze(dim=-1)

    prob = self.forward(x)
    dist = Categorical(prob)
    action = dist.sample()
    log_prob = dist.log_prob(action)
    return action,log_prob

# 替换方法
import types
model.sample_action = types.MethodType(sample_action, model)

In [ ]:
# env = CliffWalking2x3Env()
observation, _ = env.reset()
episode_number = 0
prev_x = None
xs = []
logps = []
hs = []
actions = []
ys = []
rewards = []
train_time = 0
total_sum = 0
max_times = 0
while True:
    prev_o = observation
    idxs = torch.LongTensor([observation]) # shape (1,)
    action,log_prob = model.sample_action(idxs) # shape 1,  1,
    
    action = action.item() ## 转成数
    logps.append(log_prob)
#     log_prob = log_prob.item()
    
    observation, reward, terminated, truncated, info = env.step(action)  # 更新为新的返回值
    
        # 修改奖励结构
    if reward == -100:  # 掉入悬崖
        reward = -10  # 减小惩罚力度，避免过度规避风险
    elif terminated and observation == 47:  # 到达目标状态（右下角）
        reward = 10  # 给予明显的正向奖励

#     reward = reward_shaper(prev_o,observation,terminated)
# #     print(action,reward)
#     if terminated:
#         reward = 1000
    
    rewards.append(reward*1.0)
    xs.append(prev_o)
    actions.append(action)
    total_sum += reward
    max_times += 1
    

    if terminated or truncated or max_times > 1000:
#         print(len(actions),actions[:30])
        episode_number += 1
        xs = np.vstack(xs).ravel()
        actions = np.vstack(actions)
        rewards = np.vstack(rewards).ravel()
        rewards = discount_rewards(rewards)
        rewards -= rewards.sum()
        rewards /= (rewards.std()+1e-8)

        rewards = torch.FloatTensor(rewards)
        xs = torch.LongTensor(xs)
        actions = torch.LongTensor(actions) # B,1
        
        ps = model(xs) ### B,4
        # 防止概率为0或1导致log计算爆炸
        logps = torch.cat(logps)
#         print(logps.shape,rewards.shape)
#         logps = torch.log(ps.gather(dim=-1,index=actions)).squeeze() # B,1
#         print('ps',ps.shape)
#         print('action',actions.shape)
#         print('rewards',rewards)
#         print('logps',logps)
        loss = -(logps * rewards).mean()
        #### 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print('rewards',rewards)
#         print('logps',ps,logps)
        print('total',loss,len(logps),total_sum)
        
        xs = []
        rewards = []
        logps = []
        hs = []
        actions = []
        prev_x = None
        total_sum = 0
        max_times = 0
        observation, _ = env.reset()
        
        

total tensor(103.3958, grad_fn=<NegBackward0>) 101 -108
total tensor(1605.2445, grad_fn=<NegBackward0>) 1001 -1469
total tensor(1658.8826, grad_fn=<NegBackward0>) 1001 -1523
total tensor(652.4825, grad_fn=<NegBackward0>) 426 -631
total tensor(724.8765, grad_fn=<NegBackward0>) 461 -720
total tensor(458.5815, grad_fn=<NegBackward0>) 353 -459
total tensor(1476.6172, grad_fn=<NegBackward0>) 1001 -1442
total tensor(127.8369, grad_fn=<NegBackward0>) 114 -139
total tensor(1645.8586, grad_fn=<NegBackward0>) 1001 -1586
total tensor(697.9135, grad_fn=<NegBackward0>) 524 -657
total tensor(175.7951, grad_fn=<NegBackward0>) 134 -204
total tensor(1269.9437, grad_fn=<NegBackward0>) 947 -1287
total tensor(134.5563, grad_fn=<NegBackward0>) 134 -177
total tensor(1284.7568, grad_fn=<NegBackward0>) 1001 -1343
total tensor(461.4749, grad_fn=<NegBackward0>) 378 -511
total tensor(1281.5402, grad_fn=<NegBackward0>) 1001 -1451
total tensor(1201.9324, grad_fn=<NegBackward0>) 1001 -1460
total tensor(341.2062, gr

total tensor(977.2396, grad_fn=<NegBackward0>) 1001 -1946
total tensor(973.9130, grad_fn=<NegBackward0>) 1001 -1865
total tensor(973.0195, grad_fn=<NegBackward0>) 1001 -1748
total tensor(987.3819, grad_fn=<NegBackward0>) 1001 -1595
total tensor(976.3616, grad_fn=<NegBackward0>) 1001 -1820
total tensor(984.3308, grad_fn=<NegBackward0>) 1001 -2009
total tensor(985.9248, grad_fn=<NegBackward0>) 1001 -1604
total tensor(132.9812, grad_fn=<NegBackward0>) 151 -284
total tensor(1008.0211, grad_fn=<NegBackward0>) 1001 -1496
total tensor(975.6362, grad_fn=<NegBackward0>) 1001 -1802
total tensor(975.4971, grad_fn=<NegBackward0>) 1001 -1820
total tensor(999.3065, grad_fn=<NegBackward0>) 1001 -1532
total tensor(977.8429, grad_fn=<NegBackward0>) 1001 -1721
total tensor(975.1783, grad_fn=<NegBackward0>) 1001 -1811
total tensor(984.3331, grad_fn=<NegBackward0>) 1001 -1622
total tensor(988.5807, grad_fn=<NegBackward0>) 1001 -1568
total tensor(980.2633, grad_fn=<NegBackward0>) 1001 -1937
total tensor(99

total tensor(981.0837, grad_fn=<NegBackward0>) 1001 -2009
total tensor(976.1320, grad_fn=<NegBackward0>) 1001 -1793
total tensor(693.2255, grad_fn=<NegBackward0>) 729 -1240
total tensor(991.3957, grad_fn=<NegBackward0>) 1001 -1577
total tensor(972.1302, grad_fn=<NegBackward0>) 1001 -1802
total tensor(987.2001, grad_fn=<NegBackward0>) 1001 -1622
total tensor(975.0482, grad_fn=<NegBackward0>) 1001 -1811
total tensor(478.4750, grad_fn=<NegBackward0>) 495 -718
total tensor(972.6896, grad_fn=<NegBackward0>) 1001 -1793
total tensor(974.9578, grad_fn=<NegBackward0>) 1001 -1757
total tensor(974.8975, grad_fn=<NegBackward0>) 1001 -1910
total tensor(975.0233, grad_fn=<NegBackward0>) 1001 -1838
total tensor(974.8759, grad_fn=<NegBackward0>) 1001 -1793
total tensor(975.4305, grad_fn=<NegBackward0>) 1001 -1739
total tensor(974.9070, grad_fn=<NegBackward0>) 1001 -1883
total tensor(974.7273, grad_fn=<NegBackward0>) 1001 -1784
total tensor(60.9108, grad_fn=<NegBackward0>) 75 -73
total tensor(358.5017,

total tensor(976.5488, grad_fn=<NegBackward0>) 1001 -1712
total tensor(975.1663, grad_fn=<NegBackward0>) 1001 -1892
total tensor(975.4703, grad_fn=<NegBackward0>) 1001 -1730
total tensor(974.4194, grad_fn=<NegBackward0>) 1001 -1874
total tensor(980.9238, grad_fn=<NegBackward0>) 1001 -1667
total tensor(977.2264, grad_fn=<NegBackward0>) 1001 -1928
total tensor(977.2518, grad_fn=<NegBackward0>) 1001 -1955
total tensor(883.0905, grad_fn=<NegBackward0>) 924 -1597
total tensor(974.9518, grad_fn=<NegBackward0>) 1001 -1820
total tensor(976.3486, grad_fn=<NegBackward0>) 1001 -1937
total tensor(974.9873, grad_fn=<NegBackward0>) 1001 -1901
total tensor(974.9828, grad_fn=<NegBackward0>) 1001 -1784
total tensor(974.6449, grad_fn=<NegBackward0>) 1001 -1892
total tensor(979.7445, grad_fn=<NegBackward0>) 1001 -1964
total tensor(983.9647, grad_fn=<NegBackward0>) 1001 -1622
total tensor(976.7778, grad_fn=<NegBackward0>) 1001 -1730
total tensor(975.0523, grad_fn=<NegBackward0>) 1001 -1766
total tensor(97

total tensor(975.0208, grad_fn=<NegBackward0>) 1001 -1748
total tensor(837.1588, grad_fn=<NegBackward0>) 865 -1349
total tensor(975.1447, grad_fn=<NegBackward0>) 1001 -1883
total tensor(974.0826, grad_fn=<NegBackward0>) 1001 -1820
total tensor(976.2225, grad_fn=<NegBackward0>) 1001 -1793
total tensor(975.8296, grad_fn=<NegBackward0>) 1001 -1910
total tensor(974.6727, grad_fn=<NegBackward0>) 1001 -1856
total tensor(976.4041, grad_fn=<NegBackward0>) 1001 -1712
total tensor(849.2159, grad_fn=<NegBackward0>) 883 -1430
total tensor(975.1460, grad_fn=<NegBackward0>) 1001 -1892
total tensor(976.4741, grad_fn=<NegBackward0>) 1001 -1892
total tensor(982.2851, grad_fn=<NegBackward0>) 1001 -2027
total tensor(975.7116, grad_fn=<NegBackward0>) 1001 -1901
total tensor(975.3069, grad_fn=<NegBackward0>) 1001 -1748
total tensor(986.2908, grad_fn=<NegBackward0>) 1001 -2063
total tensor(974.7208, grad_fn=<NegBackward0>) 1001 -1883
total tensor(976.2936, grad_fn=<NegBackward0>) 1001 -1892
total tensor(990

...

total tensor(988.8380, grad_fn=<NegBackward0>) 1001 -2117
total tensor(410.0955, grad_fn=<NegBackward0>) 431 -879
total tensor(979.5430, grad_fn=<NegBackward0>) 1001 -1991
total tensor(974.3466, grad_fn=<NegBackward0>) 1001 -1793
total tensor(980.4734, grad_fn=<NegBackward0>) 1001 -1667
total tensor(974.2107, grad_fn=<NegBackward0>) 1001 -1802
total tensor(975.0602, grad_fn=<NegBackward0>) 1001 -1883
total tensor(975.3815, grad_fn=<NegBackward0>) 1001 -1892
total tensor(981.2162, grad_fn=<NegBackward0>) 1001 -1658
total tensor(975.5544, grad_fn=<NegBackward0>) 1001 -1748
total tensor(976.0004, grad_fn=<NegBackward0>) 1001 -1919
total tensor(978.5529, grad_fn=<NegBackward0>) 1001 -1973
total tensor(601.2280, grad_fn=<NegBackward0>) 626 -966
total tensor(978.1953, grad_fn=<NegBackward0>) 1001 -1694
total tensor(178.3441, grad_fn=<NegBackward0>) 200 -342
total tensor(975.0678, grad_fn=<NegBackward0>) 1001 -1883
total tensor(974.2437, grad_fn=<NegBackward0>) 1001 -1847
total tensor(976.185

total tensor(986.3732, grad_fn=<NegBackward0>) 1001 -1613
total tensor(974.2902, grad_fn=<NegBackward0>) 1001 -1829
total tensor(992.8145, grad_fn=<NegBackward0>) 1001 -2162
total tensor(979.7141, grad_fn=<NegBackward0>) 1001 -1991
total tensor(980.0886, grad_fn=<NegBackward0>) 1001 -2000
total tensor(978.8165, grad_fn=<NegBackward0>) 1001 -1685
total tensor(986.4881, grad_fn=<NegBackward0>) 1001 -1613
total tensor(433.2430, grad_fn=<NegBackward0>) 462 -784
total tensor(702.5507, grad_fn=<NegBackward0>) 732 -1162
total tensor(976.0132, grad_fn=<NegBackward0>) 1001 -1919
total tensor(978.5974, grad_fn=<NegBackward0>) 1001 -1973
total tensor(974.1021, grad_fn=<NegBackward0>) 1001 -1802
total tensor(974.6339, grad_fn=<NegBackward0>) 1001 -1865
total tensor(980.2742, grad_fn=<NegBackward0>) 1001 -2000
total tensor(980.7074, grad_fn=<NegBackward0>) 1001 -2009
total tensor(974.6647, grad_fn=<NegBackward0>) 1001 -1865
total tensor(980.1186, grad_fn=<NegBackward0>) 1001 -2000
total tensor(976.

total tensor(974.2487, grad_fn=<NegBackward0>) 1001 -1829
total tensor(433.7807, grad_fn=<NegBackward0>) 462 -838
total tensor(976.0580, grad_fn=<NegBackward0>) 1001 -1730
total tensor(983.2385, grad_fn=<NegBackward0>) 1001 -1640
total tensor(974.5844, grad_fn=<NegBackward0>) 1001 -1865
total tensor(989.5468, grad_fn=<NegBackward0>) 1001 -2126
total tensor(976.5922, grad_fn=<NegBackward0>) 1001 -1721
total tensor(976.9930, grad_fn=<NegBackward0>) 1001 -1712
total tensor(974.3488, grad_fn=<NegBackward0>) 1001 -1847
total tensor(492.6765, grad_fn=<NegBackward0>) 522 -862
total tensor(983.8633, grad_fn=<NegBackward0>) 1001 -2054
total tensor(983.1164, grad_fn=<NegBackward0>) 1001 -2045
total tensor(975.9874, grad_fn=<NegBackward0>) 1001 -1730
total tensor(975.5364, grad_fn=<NegBackward0>) 1001 -1748
total tensor(990.5518, grad_fn=<NegBackward0>) 1001 -1586
total tensor(974.6064, grad_fn=<NegBackward0>) 1001 -1784
total tensor(974.6444, grad_fn=<NegBackward0>) 1001 -1784
total tensor(974.1

total tensor(977.6623, grad_fn=<NegBackward0>) 1001 -1703
total tensor(976.5566, grad_fn=<NegBackward0>) 1001 -1739
total tensor(975.8550, grad_fn=<NegBackward0>) 1001 -1919
total tensor(975.5115, grad_fn=<NegBackward0>) 1001 -1748
total tensor(974.2460, grad_fn=<NegBackward0>) 1001 -1847
total tensor(973.9894, grad_fn=<NegBackward0>) 1001 -1847
total tensor(974.1763, grad_fn=<NegBackward0>) 1001 -1856
total tensor(985.1525, grad_fn=<NegBackward0>) 1001 -1622
total tensor(980.0492, grad_fn=<NegBackward0>) 1001 -1991
total tensor(998.1853, grad_fn=<NegBackward0>) 1001 -2216
total tensor(975.6647, grad_fn=<NegBackward0>) 1001 -1910
total tensor(997.3271, grad_fn=<NegBackward0>) 1001 -1550
total tensor(974.3301, grad_fn=<NegBackward0>) 1001 -1874
total tensor(978.4960, grad_fn=<NegBackward0>) 1001 -1694
total tensor(341.8052, grad_fn=<NegBackward0>) 368 -609
total tensor(974.5515, grad_fn=<NegBackward0>) 1001 -1775
total tensor(974.8498, grad_fn=<NegBackward0>) 1001 -1892
total tensor(92.

total tensor(974.7244, grad_fn=<NegBackward0>) 1001 -1865
total tensor(975.4954, grad_fn=<NegBackward0>) 1001 -1901
total tensor(404.5090, grad_fn=<NegBackward0>) 432 -727
total tensor(978.2018, grad_fn=<NegBackward0>) 1001 -1964
total tensor(977.2603, grad_fn=<NegBackward0>) 1001 -1946
total tensor(32.3621, grad_fn=<NegBackward0>) 50 -48
total tensor(993.5795, grad_fn=<NegBackward0>) 1001 -1568
total tensor(319.7371, grad_fn=<NegBackward0>) 335 -720
total tensor(984.4822, grad_fn=<NegBackward0>) 1001 -2063
total tensor(976.0131, grad_fn=<NegBackward0>) 1001 -1919
total tensor(974.9508, grad_fn=<NegBackward0>) 1001 -1766
total tensor(974.1773, grad_fn=<NegBackward0>) 1001 -1820
total tensor(974.3829, grad_fn=<NegBackward0>) 1001 -1811
total tensor(974.2792, grad_fn=<NegBackward0>) 1001 -1802
total tensor(974.6423, grad_fn=<NegBackward0>) 1001 -1775
total tensor(993.5497, grad_fn=<NegBackward0>) 1001 -1568
total tensor(974.6021, grad_fn=<NegBackward0>) 1001 -1865
total tensor(975.0590, 

In [ ]:
np.log(0.6695)

In [ ]:
for k in model.parameters():
    print(k)

In [ ]:
torch.save(model.state_dict(),'model.pt')

In [ ]:
!ls model.pt

In [ ]:
env = gym.make("Pong-v4",render_mode="human")
observation, _ = env.reset()

In [31]:
model = MLP(D,1)

In [32]:
model.load_state_dict(torch.load('model.pt'))

In [33]:
prev_x = None
while True:
    current_x = prepro(observation)
    gap = np.zeros_like(current_x) if prev_x is None else current_x - prev_x
    prob = model(torch.from_numpy(gap))
    action = 2 if prob < 0.5 else 3
    observation, reward, terminated, truncated, info = env.step(action)  # 更新为新的返回值
    prev_x = current_x
    if terminated:
        observation, _ = env.reset()
        prev_x = None

In [34]:
env.close()

In [26]:
import numpy as np
print(11)

class CliffWalking2x3Env:
    def __init__(self):
        self.shape = (2, 3)  # 2行3列
        self.start_state = 3  # 起点在第二行第一列（状态3）
        self.cliff_state = 1  # 悬崖在第一行第二列（状态1）
        self.goal_state = 5  # 终点在第二行第三列（状态5）
        self.current_state = self.start_state
        
        # 动作空间：0=上，1=右，2=下，3=左
        self.action_space = 4
        self.observation_space = 6  # 状态空间大小
        
    def reset(self):
        """重置环境到初始状态"""
        self.current_state = self.start_state
        return self.current_state,{}
    
    def state_to_pos(self, state):
        """将状态编号转换为坐标"""
        row = state // 3
        col = state % 3
        return (row, col)
    
    def pos_to_state(self, row, col):
        """将坐标转换为状态编号"""
        return row * 3 + col
    
    def step(self, action):
        """执行动作并返回新状态、奖励、是否终止"""
        row, col = self.state_to_pos(self.current_state)
        new_row, new_col = row, col
        
        # 计算新坐标
        if action == 0:    # 上
            new_row = max(row - 1, 0)
        elif action == 1:  # 右
            new_col = min(col + 1, 2)
        elif action == 2:  # 下
            new_row = min(row + 1, 1)
        elif action == 3:  # 左
            new_col = max(col - 1, 0)
        
        # 检查是否越界（此逻辑已包含在max/min中，可省略）
        new_state = self.pos_to_state(new_row, new_col)
        reward = -1  # 默认每步-1奖励
        done = False
        
        # 判断是否掉下悬崖或到达终点
        if new_state == self.cliff_state:
            reward = -100
            self.current_state = self.start_state  # 回到起点
        elif new_state == self.goal_state:
            reward = 10
            done = True
            self.current_state = new_state
        else:
            self.current_state = new_state
        
        return self.current_state, reward, done,done, {}
    
    def render(self):
        """可视化当前状态"""
        grid = [
            ['S', 'C', 'S'],
            ['S', 'S', 'G']
        ]
        row, col = self.state_to_pos(self.current_state)
        grid[row][col] = 'A'  # 用A表示智能体位置
        print(f"+---+---+---+")
        print(f"|{grid[0][0]}|{grid[0][1]}|{grid[0][2]}|")
        print(f"+---+---+---+")
        print(f"|{grid[1][0]}|{grid[1][1]}|{grid[1][2]}|")
        print(f"+---+---+---+")
        print()


# env = CliffWalking2x3Env()
# state = env.reset()
# done = False

# print("初始状态：")
# env.render()

# # 示例动作序列：右→右→上→右→下→右
# actions = [1, 1, 0, 1, 2, 1]

# for action in actions:
#     next_state, reward, done, _,_ = env.step(action)
#     print(f"执行动作 {action} 后:")
#     env.render()
#     print(f"奖励: {reward}, 终止: {done}\n")
#     if done:
#         break

11
